Project : Personalized Recipe Recommender

Objective
Create an application that suggests recipes based on user preferences and dietary restrictions. Users can input ingredients they have on hand or specify their dietary needs, and the application will recommend suitable recipes.

Key Features

Ingredient Input: Users can enter available ingredients.

Dietary Preferences: Allow users to specify dietary restrictions (e.g., vegetarian, gluten-free).

Recipe Fetching: Use an API to fetch recipes that match the criteria.

Learning Outcomes
Learn about text processing and matching techniques.
Understand how to work with APIs to retrieve data.
Gain experience in handling user input and providing personalized outputs.

**Conclusion**
This project is engaging and allows users to explore cooking while learning essential NLP concepts. It also introduces you to working with APIs, enhancing your programming skills.

In [6]:
# making a project on Project Idea: Personalized Recipe Recommender



In [7]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import streamlit as st
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Load dataset
df = pd.read_csv('/content/Food Ingredients and Recipe Dataset with Image Name Mapping.csv')
print(df.head())

   Unnamed: 0                                              Title  \
0           0  Miso-Butter Roast Chicken With Acorn Squash Pa...   
1           1                    Crispy Salt and Pepper Potatoes   
2           2                        Thanksgiving Mac and Cheese   
3           3                 Italian Sausage and Bread Stuffing   
4           4                                       Newton's Law   

                                         Ingredients  \
0  ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...   
1  ['2 large egg whites', '1 pound new potatoes (...   
2  ['1 cup evaporated milk', '1 cup whole milk', ...   
3  ['1 (¾- to 1-pound) round Italian loaf, cut in...   
4  ['1 teaspoon dark brown sugar', '1 teaspoon ho...   

                                        Instructions  \
0  Pat chicken dry with paper towels, season all ...   
1  Preheat oven to 400°F and line a rimmed baking...   
2  Place a rack in middle of oven; preheat to 400...   
3  Preheat oven to 350°F with 

In [11]:
# extracting more about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           13501 non-null  int64 
 1   Title                13496 non-null  object
 2   Ingredients          13501 non-null  object
 3   Instructions         13493 non-null  object
 4   Image_Name           13501 non-null  object
 5   Cleaned_Ingredients  13501 non-null  object
dtypes: int64(1), object(5)
memory usage: 633.0+ KB


In [12]:
# describing the dataset
df.describe()


,Unnamed: 0
count,13501.000000
mean,6750.000000
std,3897.547327
min,0.000000
25%,3375.000000
50%,6750.000000
75%,10125.000000
max,13500.000000


In [13]:
# Doing EDA on data
df.isnull().sum()


,0
Unnamed: 0,0
Title,5
Ingredients,0
Instructions,8
Image_Name,0
Cleaned_Ingredients,0


In [14]:
# dropping null values data from the dataframe
df.dropna(inplace=True)


In [15]:
# checking data again
df.isnull().sum()

,0
Unnamed: 0,0
Title,0
Ingredients,0
Instructions,0
Image_Name,0
Cleaned_Ingredients,0


In [16]:
# check for duplicate values
df.duplicated().sum()

0

In [17]:
# hence there are no duplicate or null values in the data anymore

In [18]:
# Preprocess ingredients data
df['Ingredients'] = df['Ingredients'].str.lower().str.split(', ')

In [19]:
df.head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[['1 (3½–4-lb.) whole chicken', '2¾ tsp. koshe...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"[['2 large egg whites', '1 pound new potatoes ...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"[['1 cup evaporated milk', '1 cup whole milk',...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"[['1 (¾- to 1-pound) round italian loaf, cut i...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"[['1 teaspoon dark brown sugar', '1 teaspoon h...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [20]:
# function to get the user input for the ingredients

def get_user_input():
    while True:
        user_input = input("Enter the ingredients you have (comma-separated): ")

        # Check if the input is empty
        if not user_input.strip():
            print("You didn't enter anything. Please provide some ingredients.")
            continue

        # Split the input by commas and strip whitespace
        Ingredients = [Ingredient.strip().lower() for Ingredient in user_input.split(',')]

        # Check if any ingredient is empty after stripping
        if any(Ingredient == '' for Ingredient in Ingredients):
            print("Please enter valid ingredients. Make sure they are comma-separated.")
            continue

        return Ingredients

In [21]:
#the recipe recommendation function

def recommend_recipes(user_ingredients):
    recommended_recipes = []

    # Iterate through each recipe in the DataFrame
    for index, row in df.iterrows():
        # Check if any user ingredient is in the recipe's ingredients
        # Convert recipe ingredients to lowercase for comparison
        recipe_ingredients_lower = [Ingredient.lower() for Ingredient in row['Ingredients']]
        if any(Ingredient in recipe_ingredients_lower for Ingredient in user_ingredients):
            recommended_recipes.append(row[['Title', 'Ingredients']])

    return pd.DataFrame(recommended_recipes)

In [22]:
# main function to call other functions
# main function to call other functions
# main function to call other functions
# main function to call other functions

def main():
    print("Welcome to the Personalized Recipe Recommender!")

    user_ingredients = get_user_input()

    recommendations = recommend_recipes(user_ingredients)

    print("\nRecommended Recipes:")
    print("=" * 30)  # Separator line
    if not recommendations.empty:
        for index, row in recommendations.iterrows():
            print(f"\nRecipe Name: {row['Title']}")
            print("Ingredients:")
            ingredients_list = row['Ingredients'] # Assuming 'Ingredients' is already a list of ingredients
            for Ingredient in ingredients_list:
                print(f" - {Ingredient.strip()}")  # Correctly access 'Ingredient'
            print("-" * 30)  # Separator line between recipes
    else:
        print("No recipes found with the provided ingredients.")

        # Suggest adding missing ingredients
        # Convert the 'Ingredients' column to lowercase and split before creating the set
        all_ingredients = [ingredient.lower() for sublist in df['Ingredients'] for ingredient in sublist]

        missing_ingredients = set(user_ingredients) - set(all_ingredients)

        if missing_ingredients:
            print("\nYou can consider adding these new ingredients:")
            for ingredient in missing_ingredients:
                print(f" - {ingredient}") # Correctly access 'ingredient'
                add_more = input("\nWould you like to add these new ingredients to your list? (yes/no): ").strip().lower()

            if add_more == 'yes':
                # Here you could implement functionality to save these new ingredients or use them later.
                print("New ingredients added to your list!")
                # For demonstration purposes, we just display them.
                print(f"Added Ingredients: {', '.join(missing_ingredients)}")
            else:
                print("No new ingredients added.")







In [23]:
#run

main()

Welcome to the Personalized Recipe Recommender!
Enter the ingredients you have (comma-separated): onion,capsicum,mushroom,chicken

Recommended Recipes:

Recipe Name: Veselka's Famous Borscht
Ingredients:
 - ['3 pounds (10 to 12) small beets
 - scrubbed thoroughly but not peeled'
 - '9 tablespoons white vinegar'
 - 'one 2-pound boneless pork butt
 - halved'
 - '8 cups beef
 - chicken
 - or vegetable stock'
 - '1 bay leaf'
 - '1 teaspoon whole allspice berries'
 - '1 tablespoon whole black peppercorns'
 - '3 large carrots
 - peeled and sliced'
 - '3 large celery stalks
 - sliced'
 - '1 small head of green cabbage (about ¾ to 1 pound)
 - shredded (about 4 cups)'
 - '2 medium idaho potatoes
 - peeled and cut into ½-inch dice'
 - 'one 15-ounce can lima beans
 - drained and rinsed'
 - 'salt']
------------------------------

Recipe Name: Braised Celery With Lentils and Garlic
Ingredients:
 - ['¼ cup plus 1 tbsp. extra-virgin olive oil'
 - '1 bunch celery
 - leaves reserved
 - stalks sliced ½”